# importing Libraries

In [ ]:
from time import time
from pynq import Overlay
import numpy as np
from pynq import allocate

ol_adds = Overlay('design_1.bit')
dma_ip = ol_adds.axi_dma_0



# Function to join the numbers we want to add in a single element to be sent to the adder then

In [ ]:
def concatenate_to_64bit(int1: int, int2: int) -> int:
    """
    Concatenate two 32-bit integers into a 64-bit integer.
    
    Args:
        int1 (int): The first 32-bit integer (will occupy bits 63 to 32).
        int2 (int): The second 32-bit integer (will occupy bits 31 to 0).
    
    Returns:
        int: The concatenated 64-bit integer.
    """
    # Ensure the inputs are treated as 32-bit integers
    int1 &= 0xFFFFFFFF
    int2 &= 0xFFFFFFFF
    
    # Shift the first integer to the left by 32 bits and OR with the second integer
    result = (int1 << 32) | int2
    return result

## Define Add Function (in reality it is a transmision and receive function)


In [ ]:
def adds_data():
    dma_ip.sendchannel.transfer(in_buffer)
    dma_ip.recvchannel.transfer(out_buffer)
    dma_ip.sendchannel.wait()
    dma_ip.recvchannel.wait()

In [ ]:
my_data = []
for i in range(30):
    my_data.append(concatenate_to_64bit(i,i))

num_data = len(my_data)
print (my_data) #numbers to be added cannot be recognized in decimal format
"""
We allocate the buffers in memory, note that the input buffer is 64 bits and the output is 32 bits.
To avoid overflow we should have designed the adder to have an output width of 32+1 = 33 bit. I have not done this to keep 
things simple.
"""
in_buffer = allocate(shape=(num_data,), dtype=np.uint64)
out_buffer = allocate(shape=(num_data,), dtype=np.uint32)



# Copying DATA into the input Buffer

In [ ]:
for i in range(num_data):
    in_buffer[i] = my_data[i]

# Peforming addition and printing output

In [ ]:
adds_data()

#print the summation output
for i in range(num_data):
    print(out_buffer[i])


# Free the memory at the end.

In [ ]:
del in_buffer
del out_buffer